In [50]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [51]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10598065586811741176
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14982706565213778576
physical_device_desc: "device: XLA_CPU device"
]


In [52]:
import os
import math
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [53]:
tf.__version__

'2.1.0'

In [54]:
# return gray image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


In [55]:
# # training targets, you can use your custome csv file if you already created it with "training-sample.py"
# targets = pd.Series.from_csv('../Downloads/jester-v1-train.csv',header=None,sep = ";").to_dict()
# targets[34870]
# # validation targets, you can use your custome csv file if you already created it with "validation-sample.py"
# targets_validation = pd.Series.from_csv('../Downloads/jester-v1-validation.csv',header=None,sep = ";").to_dict()
# targets_validation[9223]

training_labels_df = pd.read_csv('../Downloads/jester/jester-v1-train.csv' , header=None,sep = ";")
validation_labels_df = pd.read_csv('../Downloads/jester/jester-v1-validation.csv' , header=None,sep = ";")

In [56]:
targets_name = [
    "Thumb Down",
    "Stop Sign",
    "Sliding Two Fingers Left",
    "Sliding Two Fingers Right",
    "No gesture",
    "Thumb Up"
    ]

In [57]:
def select_column(col_name):
    if col_name in targets_name:
        return 1
    else:
        return 0
    
training_labels_df['filtered'] = training_labels_df.iloc[:,1].apply(select_column)
validation_labels_df['filtered'] = validation_labels_df.iloc[:,1].apply(select_column)

In [58]:
training_labels_df.columns = ['class_no' , 'class_name' , 'filtered']

In [59]:
validation_labels_df.columns = ['class_no' , 'class_name' , 'filtered']

In [60]:
training_labels_df.head()

,class_no,class_name,filtered
0,34870,Drumming Fingers,0
1,56557,Sliding Two Fingers Right,1
2,129112,Sliding Two Fingers Down,0
3,63861,Pulling Two Fingers In,0
4,131717,Sliding Two Fingers Up,0


In [61]:
validation_labels_df.head()

,class_no,class_name,filtered
0,9223,Thumb Up,1
1,107090,Pushing Two Fingers Away,0
2,42920,Swiping Left,0
3,106485,Thumb Down,1
4,142201,Rolling Hand Backward,0


In [62]:
ROOT_DIR = '../Downloads/jester/20bn-jester-v1'
import os
import shutil


def filter_out_dataset(df, targets_name, train=True):
    
    try:
        for class_name in targets_name:
            if train:
                os.makedirs(os.path.join('jester','train' , class_name))
            else:
                os.makedirs(os.path.join('jester','valid' , class_name))
    except FileExistsError:
            pass
        
    for i in range(df.shape[0]):
        if df.loc[i,'filtered'] == 1:
            # print(df.loc[i,'class_no'] ,df.loc[i,'class_name'] )
            source = os.path.join(ROOT_DIR , str(df.loc[i,'class_no'] ) )
            if train:
                destination = os.path.join('jester','train' , df.loc[i,'class_name'])
            else:
                destination = os.path.join('jester','valid' , df.loc[i,'class_name'])
            # print(source , destination)
            shutil.move( source , destination )
            
        
filter_out_dataset(training_labels_df,targets_name,True)
filter_out_dataset(validation_labels_df,targets_name,False)


In [80]:
# return gray image
from PIL import Image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

# Resize frames
def resize_frame(frame):
    frame = cv2.imread(frame)
    frame = cv2.resize(frame, (64, 64))
    return frame


def clean_and_preprocess_dataset(root_dir , targets_name , train):
    
    for class_name in targets_name:
        if train:
            base_path =  os.path.join(root_dir, 'train', class_name)
            folder_list = os.listdir(base_path)
            
        else:
            base_path = os.path.join(root_dir, 'valid', class_name)
            folder_list = os.listdir(base_path)
            
        #print(folder_list)
        
        for each_folder in folder_list:
            folder_path = os.path.join(base_path,each_folder)
            
            frames = os.listdir(folder_path)
            new_folder_path = folder_path.replace('jester', 'jester_processed')
            
            try:
                os.makedirs(new_folder_path)
            except FileExistsError:
                pass
            
            frame_length = len(frames)
            frames = sorted(frames)
            if frame_length < 30:
                for _ in range(30- frame_length):
                    frames.append(frames[frame_length-1])
            else:
                frames = frames[:30]
                
            for each_frame in frames:
                frames_path = os.path.join(folder_path,each_frame)
#                 im = Image.open(frames_path)
#                 imResize = im.resize((64,64), Image.ANTIALIAS)
#                 imgBlack = imResize.convert('LA')
#                 frames_path_name = frames_path.split('.')[0]
#                 imgBlack.save(frames_path)
                # print(frames_path_name)
                resized = resize_frame(frames_path)
                gray = rgb2gray(resized)
                new_frames_path = frames_path.replace('jester', 'jester_processed')
                #print(new_frames_path)
                cv2.imwrite(new_frames_path, gray)
                #print(gray.shape)
                  
        

clean_and_preprocess_dataset('jester', targets_name, True)
clean_and_preprocess_dataset('jester', targets_name, False)

In [2]:
targets_dict = {
    "Thumb Down":0,
    "Stop Sign":1,
    "Sliding Two Fingers Left":2,
    "Sliding Two Fingers Right":3,
    "No gesture":4,
    "Thumb Up":5
    }



In [12]:
import os , cv2 
import numpy as np

def get_total_samples(datadir , targets_dict , train):
    counter = 0
    for class_name, label in targets_dict.items():
        if train:
            base_path =  os.path.join(datadir, 'train', class_name)
            folder_list = os.listdir(base_path)

        else:
            base_path = os.path.join(datadir, 'valid', class_name)
            folder_list = os.listdir(base_path)

        #print(folder_list)

        for each_folder in folder_list:
            counter += 1
    return counter

def prepare_numpy_array(datadir , targets_dict , train):
    
    
    
    n = get_total_samples(datadir , targets_dict , train)
    
    X_data = np.empty((n,30,64,64) , dtype=np.uint8)
    X_labels = np.empty((n))
    
    cn = 0
    for class_name, label in targets_dict.items():
        if train:
            base_path =  os.path.join(datadir, 'train', class_name)
            folder_list = os.listdir(base_path)
            
        else:
            base_path = os.path.join(datadir, 'valid', class_name)
            folder_list = os.listdir(base_path)
            
        #print(folder_list)
        
        for each_folder in folder_list:
            folder_path = os.path.join(base_path,each_folder)
            
            frames = os.listdir(folder_path)
            frames = sorted(frames)    
            
            X_labels[cn]= label
            
            single_sample = np.empty( (30,64,64), dtype=np.uint8)
            # print(single_sample.shape)
            for f, each_frame in enumerate(frames):
                frames_path = os.path.join(folder_path,each_frame)
                single_frame = cv2.imread(frames_path)
                # print(single_frame.dtype)
                single_sample[f,:,:] = single_frame[:,:,0]
                # print(frames_path)r
                
            X_data[cn,:,:,:] = single_sample
            cn += 1
            
        
                    
    return X_data, X_labels
                
                               
X_train , y_train = prepare_numpy_array('jester_processed' , targets_dict , True)
X_valid , y_valid = prepare_numpy_array('jester_processed' , targets_dict , False)

# print(y_train)

In [17]:
np.save('jester_X_train', X_train)
np.save('jester_y_train', y_train)
np.save('jester_X_valid', X_valid)
np.save('jester_y_valid', y_valid)


In [18]:
X_train.mean()

107.56613544380178